# TimesFM 2.5 Quantile Forecasting + Covariates for Inventory Planning

This notebook demonstrates using TimesFM 2.5's quantile head with covariates for inventory decisions:

- Covariate integration: static (store, product group) + dynamic (month, week)
- Quantile adjustments: xreg adjustments applied to ALL quantile levels (P10–P90)
- Service-level policies: target ~τ cycle service with period coverage
- Newsvendor optimization: cost-optimal quantile via Cu/(Cu+Co)
- Calibration-ready outputs

Enhancement: Combines `forecast_with_covariates()` + `use_continuous_quantile_head=True`

Data: VN2 Inventory Planning Challenge

Reference: see `quantile.md` for theory and context.


In [1]:
# Setup
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# TimesFM
import torch
timesfm_path = Path("..").resolve()
if str(timesfm_path) not in sys.path:
    sys.path.insert(0, str(timesfm_path))
import timesfm

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


PyTorch: 2.8.0
CUDA available: False


In [2]:
# Load VN2 data (robust)
DATA_DIR = Path("../../vn2inventory/data").resolve()

sales_df = pd.read_csv(DATA_DIR / "Week 0 - 2024-04-08 - Sales.csv")
master_df = pd.read_csv(DATA_DIR / "Week 0 - Master.csv")

# Strip whitespace in source columns
sales_df.columns = sales_df.columns.str.strip()
master_df.columns = master_df.columns.str.strip()

# Melt to long
id_cols = ["Store", "Product"]
assert set(id_cols).issubset(sales_df.columns), f"Missing id columns in sales_df: {set(id_cols) - set(sales_df.columns)}"

sales_long = sales_df.melt(id_vars=id_cols, var_name="date", value_name="sales_qty")

# Normalize columns and types
sales_long.columns = sales_long.columns.str.strip()
sales_long["date"] = pd.to_datetime(sales_long["date"], errors="coerce")
sales_long["sales_qty"] = pd.to_numeric(sales_long["sales_qty"], errors="coerce").fillna(0.0)
sales_long["Store"] = pd.to_numeric(sales_long["Store"], errors="coerce").fillna(0).astype(int)
sales_long["Product"] = pd.to_numeric(sales_long["Product"], errors="coerce").fillna(0).astype(int)

# Temporal features
sales_long["week_of_year"] = sales_long["date"].dt.isocalendar().week
sales_long["month"] = sales_long["date"].dt.month.astype(int)
sales_long["quarter"] = sales_long["date"].dt.quarter.astype(int)

# Merge master on Product (keep only ProductGroup, Department to avoid Store conflict)
master_cols = ["Product", "ProductGroup", "Department"]
master_subset = master_df[master_cols].drop_duplicates(subset=["Product"])
sales_long = sales_long.merge(master_subset, on="Product", how="left")

# Fill missing with 0
sales_long["ProductGroup"] = sales_long["ProductGroup"].fillna(0).astype(int)
sales_long["Department"] = sales_long["Department"].fillna(0).astype(int)

# Sort
sales_long = sales_long.sort_values(["Store", "Product", "date"]).reset_index(drop=True)

print(f"Sales data shape: {sales_long.shape}")
print(f"Columns: {list(sales_long.columns)}")
print(f"✓ Data loaded successfully")


Sales data shape: (94043, 9)
Columns: ['Store', 'Product', 'date', 'sales_qty', 'week_of_year', 'month', 'quarter', 'ProductGroup', 'Department']
✓ Data loaded successfully


In [3]:
# Load TimesFM 2.5 (quantile head)
print("Loading TimesFM 2.5 with quantile forecasting...")
model = timesfm.TimesFM_2p5_200M_torch.from_pretrained("google/timesfm-2.5-200m-pytorch")

print("Compiling with quantile head enabled...")
model.compile(
    timesfm.ForecastConfig(
        max_context=512,
        max_horizon=128,
        normalize_inputs=True,
        use_continuous_quantile_head=True,
        force_flip_invariance=True,
        infer_is_positive=True,
        fix_quantile_crossing=True,
    )
)
print("✓ Model loaded with quantile head enabled")


Loading TimesFM 2.5 with quantile forecasting...
Compiling with quantile head enabled...
✓ Model loaded with quantile head enabled


In [4]:
# Prepare subset (50 SKUs)
CONTEXT_LENGTH = 140
HORIZON = 3
TEST_START_WEEK = 140

sku_volumes = sales_long.groupby(["Store", "Product"])["sales_qty"].sum().reset_index()
sku_list = sku_volumes.sort_values("sales_qty", ascending=False).head(50)[["Store", "Product"]]

# Prepare inputs/actuals
inputs, actuals = [], []
for _, row in sku_list.iterrows():
    sku_data = sales_long[(sales_long["Store"] == row["Store"]) & (sales_long["Product"] == row["Product"])].sort_values("date")
    inputs.append(sku_data.iloc[:TEST_START_WEEK]["sales_qty"].values[-CONTEXT_LENGTH:])
    actuals.append(sku_data.iloc[TEST_START_WEEK:TEST_START_WEEK+HORIZON]["sales_qty"].values)

actuals = np.array(actuals)
print(f"Prepared {len(inputs)} SKUs for testing")


Prepared 50 SKUs for testing


In [5]:
# Prepare covariates
required_cols = {"Store", "Product", "date", "sales_qty", "month", "week_of_year", "quarter"}
missing = required_cols - set(sales_long.columns)
assert not missing, f"Missing required columns in sales_long: {missing}"

static_categorical_covariates = {"store": [], "product_group": [], "department": []}
static_numerical_covariates = {"mean_demand": [], "cv_demand": []}
dynamic_categorical_covariates = {"month": [], "week_of_year": [], "quarter": []}
dynamic_numerical_covariates = {"week_index": []}

for _, row in sku_list.iterrows():
    sku_data = sales_long[(sales_long["Store"] == row["Store"]) & (sales_long["Product"] == row["Product"])].sort_values("date")
    if sku_data.empty:
        continue
    
    # Static features
    static_categorical_covariates["store"].append(int(row["Store"]))
    static_categorical_covariates["product_group"].append(int(sku_data.iloc[0].get("ProductGroup", 0)))
    static_categorical_covariates["department"].append(int(sku_data.iloc[0].get("Department", 0)))
    
    mean_demand = float(sku_data["sales_qty"].mean())
    std_demand = float(sku_data["sales_qty"].std())
    cv_demand = (std_demand / mean_demand) if mean_demand > 0 else 0.0
    static_numerical_covariates["mean_demand"].append(mean_demand)
    static_numerical_covariates["cv_demand"].append(cv_demand)
    
    # Dynamic features
    context_data = sku_data.iloc[:TEST_START_WEEK][-CONTEXT_LENGTH:]
    horizon_data = sku_data.iloc[TEST_START_WEEK:TEST_START_WEEK+HORIZON]
    combined = pd.concat([context_data, horizon_data])
    
    dynamic_categorical_covariates["month"].append(combined["month"].astype(int).tolist())
    dynamic_categorical_covariates["week_of_year"].append(combined["week_of_year"].astype(int).tolist())
    dynamic_categorical_covariates["quarter"].append(combined["quarter"].astype(int).tolist())
    dynamic_numerical_covariates["week_index"].append(list(range(len(combined))))

print(f"✓ Covariates prepared for {len(inputs)} SKUs")


✓ Covariates prepared for 50 SKUs


In [6]:
# Forecast with covariates + quantiles
print("Generating quantile forecasts with covariates...")
combined_forecast, xreg_forecast, quantile_forecast_list = model.forecast_with_covariates(
    horizon=HORIZON,
    inputs=inputs,
    static_categorical_covariates=static_categorical_covariates,
    static_numerical_covariates=static_numerical_covariates,
    dynamic_categorical_covariates=dynamic_categorical_covariates,
    dynamic_numerical_covariates=dynamic_numerical_covariates,
    xreg_mode="xreg + timesfm",
)

point_forecast = np.array(combined_forecast)
quantile_forecast = np.array(quantile_forecast_list)

print(f"Point forecast (with covariates): {point_forecast.shape}")
print(f"Quantile forecast (with covariates): {quantile_forecast.shape}")
assert quantile_forecast.shape[-1] == 10, "Expected last dim = 10 ([mean, P10..P90])"
print("✓ Quantile tensor format validated")


Generating quantile forecasts with covariates...
Point forecast (with covariates): (50, 3)
Quantile forecast (with covariates): (50, 3, 10)
✓ Quantile tensor format validated
